In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# This blocks allows me to grab all of the correlation matrices from our LS8 folder.

src_file = '/content/gdrive/My Drive/test_FL_DEP'
avg_set = []
im_set = []
subdirectories = []
for fn, subdir, dir in os.walk(src_file):
  for folder in subdir:
    subdirectories += [folder]
  for file in dir:
    if '_bandAverages_B1-B8A' in file:
      avg_set += [file]
    #default is 9:47 if your prefix is 3 characters use 10:48
      im_set += [file[10:48]+"_outputs"]
    # For Landsat 8
    # if '_bandAverages_B1-B5' in file:
    #   avg_set += [file]

In [ ]:
im_set

[]

In [ ]:
avg_waterPixel_dict = {}
tot_water = {}
for i in range(len(avg_set)):
  # Read in our csv.
  avg_csv = pd.read_csv(f"{src_file}/{im_set[i]}/{avg_set[i]}", delimiter = ',')

  # Grab out our correlation matrix.
  avg_mat = avg_csv['band_values'].tolist()[0].strip('[ ]').split(',')
  out = []
  for element in avg_mat:
    out += [float(element)]

  # Grab out our pixel value.
  pixel_val = avg_csv['pixel_total'][0]

  # Pass both of these values to their respective dictionaries.
  avg_waterPixel_dict[f'{avg_set[i]}'] = out
  tot_water[f'{avg_set[i]}'] = pixel_val

band_values = list(avg_waterPixel_dict.values())

In [ ]:
# This is the total number of water pixels.
total_w = sum(tot_water.values())

weighted_arrays = {}
# For valid pixels in water_pixels
i = 0
for val_group in band_values:

  # What fraction does each set of pixels account for the total?
  weight = tot_water[avg_set[i]] / total_w

  # Apply that fraction to each weight's respective correlation matrix.
  weighted_arrays[i] = [item * weight for item in val_group]

  i+=1

In [ ]:
# Get back our weighted matrix.
pd.DataFrame(weighted_arrays.values()).sum(axis = 0).to_csv('/content/gdrive/My Drive/test_FL_DEP/s2_avgWeightedBandValues.csv')

# For Landsat 8
# pd.DataFrame(weighted_arrays.values()).sum(axis = 0).to_csv('/content/gdrive/My Drive/l8_avgWeightedBandValues.csv')